In [63]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


# restore all of our data structures
data = pickle.load( open( "training_data.parquet", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']
print(classes)
print(len(train_x[0]))
print(len(train_y[0]))

['entertainment_movies', 'entertainment_music', 'flight_assistant', 'goodbye', 'greeting', 'options', 'order_dessert', 'order_drinks', 'order_food', 'tellmejoke', 'thanks', 'weather_search']
86
12


### Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax

In [64]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

### Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

In [65]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


### Fitting and saving the model 

In [66]:

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
12/12 [==============================] - 0s 1ms/step - loss: 2.4843 - accuracy: 0.1250
Epoch 2/200
12/12 [==============================] - 0s 883us/step - loss: 2.3858 - accuracy: 0.1071
Epoch 3/200
12/12 [==============================] - 0s 1ms/step - loss: 2.3659 - accuracy: 0.1429
Epoch 4/200
12/12 [==============================] - 0s 1ms/step - loss: 2.2870 - accuracy: 0.2143
Epoch 5/200
12/12 [==============================] - 0s 1ms/step - loss: 2.2542 - accuracy: 0.1964
Epoch 6/200
12/12 [==============================] - 0s 1ms/step - loss: 2.0729 - accuracy: 0.3571
Epoch 7/200
12/12 [==============================] - 0s 1ms/step - loss: 1.9022 - accuracy: 0.3571
Epoch 8/200
12/12 [==============================] - 0s 1ms/step - loss: 1.9317 - accuracy: 0.4643
Epoch 9/200
12/12 [==============================] - 0s 1ms/step - loss: 1.6609 - accuracy: 0.5357
Epoch 10/200
12/12 [==============================] - 0s 1ms/step - loss: 1.6461 - accuracy: 0.5000
Epoch 1

Epoch 83/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1415 - accuracy: 0.9286
Epoch 84/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0387 - accuracy: 1.0000
Epoch 85/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0456 - accuracy: 1.0000
Epoch 86/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0850 - accuracy: 0.9643
Epoch 87/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1446 - accuracy: 0.9464
Epoch 88/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0505 - accuracy: 1.0000
Epoch 89/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1075 - accuracy: 0.9643
Epoch 90/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0329 - accuracy: 1.0000
Epoch 91/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0514 - accuracy: 1.0000
Epoch 92/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0321 - accuracy: 1.0000


12/12 [==============================] - 0s 962us/step - loss: 0.0587 - accuracy: 0.9643
Epoch 165/200
12/12 [==============================] - 0s 958us/step - loss: 0.0138 - accuracy: 1.0000
Epoch 166/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0491 - accuracy: 0.9821
Epoch 167/200
12/12 [==============================] - 0s 976us/step - loss: 0.0553 - accuracy: 0.9821
Epoch 168/200
12/12 [==============================] - 0s 911us/step - loss: 0.0121 - accuracy: 1.0000
Epoch 169/200
12/12 [==============================] - 0s 975us/step - loss: 0.0483 - accuracy: 0.9643
Epoch 170/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 171/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0132 - accuracy: 1.0000
Epoch 172/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 173/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0394 - accuracy: 1

In [67]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               11136     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 12)                780       
Total params: 20,172
Trainable params: 20,172
Non-trainable params: 0
_________________________________________________________________
